Global setup

In [ ]:
try:
    with open("../global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

In [ ]:
import feedparser # Parsing the RSS feed
import pandas as pd # Displaying results in a data-frame
from afinn import Afinn # Sentiment analysis package
from IPython.core.display import display#, HTML # HTML displayer
from notebooks.exercises.src.text.rsspedia import Rsspedia # Searching in Wiki for text matches using Okapi BM25
from src.text.document_retrieval.wikipedia import Wikipedia # Generic Wikipedia class

In [ ]:
# Get the Politiken RSS feed
feed = feedparser.parse("http://politiken.dk/rss/senestenyt.rss")
afinn = Afinn(language = "da")
data_titles = []
data_scores = []
# Get relevant objects from RSS feed ans store titles and scores
for i in range(len(feed["entries"])):
    data_titles.append(feed["entries"][i]["title"])
    data_scores.append(afinn.score(feed["entries"][i]["title"]))

# Dataframe
pd.set_option('display.max_colwidth', -1) # Used to display whole title (non-truncated)
df = pd.DataFrame({"Title": data_titles, "Score": data_scores}) # Creating the data frame and populating it

# Highlight the positive and negative sentiments
def highlight(s):
    if s.Score > 0:
        return ['background-color: #AAFFAA']*2
    elif s.Score < 0:
        return ['background-color: #FFAAAA']*2
    else:
        return ['background-color: #FFFFFF']*2

df = df.style.apply(highlight, axis=1)

df

In [ ]:
# Initialize the wikipedia class and (down)load the vocabulary
wikipedia = Wikipedia(language="danish", cache_directory_url=None)
# Initialize the class to search RSS titles in the Wikipedia
rsspedia = Rsspedia(wikipedia)

In [ ]:
rsspedia.search_wikipedia(data_titles)

In [ ]:
#print(data_titles[1])
#import pprint
#pprint.pprint(rsspedia.search_results)
#display(HTML(rsspedia.search_results))

In [ ]:
from ipywidgets.widgets import Accordion, FloatText, Layout, Label, VBox, HTML

list_labels = []
for i in range(len(feed["entries"])):
    list_labels.append(HTML(value = rsspedia.search_results[i]))

accordion = Accordion(children = (list_labels),)

for i in range(len(feed["entries"])):
    accordion.set_title(i, "{}. {}".format(i + 1, data_titles[i]))

display(accordion)

In [ ]:
#from dasem.wikipedia import ExplicitSemanticAnalysis
#esa = ExplicitSemanticAnalysis()
#esa.related(u"Skal Nobelprisen i litteratur reddes fra den synkende svenske skude, så flyt den til Alexandria".lower())

In [ ]:
#import nltk
#nltk.download('punkt')